In [8]:
authorization_code = "4/1AZEOvhX-3Dq8EUbegzNxnVy3TPx5xbdxO19pQznZghDDo1acJTm_fcQp6HU" #earth engine thing

In [9]:
import ee as ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output


ee.Initialize()



Successfully saved authorization token.


In [10]:
# Define button click handlers
def home_button_clicked(button):
    clear_output()
    display(home_page())

def page1_button_clicked(button):
    clear_output()
    display(page1())

def page2_button_clicked(button):
    clear_output()
    display(page2())


In [11]:
def home_page():
    # Define buttons
    home_button = widgets.Button(description='Home')
    home_button.on_click(home_button_clicked)
    page1_button = widgets.Button(description='Timelapse')
    page1_button.on_click(page1_button_clicked)
    page2_button = widgets.Button(description='Graphs')
    page2_button.on_click(page2_button_clicked)

    # Create button box
    button_box = widgets.VBox([home_button, page1_button, page2_button])

    # Create sidebar with buttons
    sidebar = widgets.VBox([button_box])

    # Additional text
    instructions = widgets.HTML(
        "<ol>"
        "<li>A pioneering company specializing in captivating timelapses of various locations. With a passion for environmental responsibility, we go beyond visuals, providing valuable carbon dioxide emission data. Our innovative approach combines stunning imagery with critical environmental insights, promoting awareness and sustainability in an engaging way.</li>"
        "</ol>"
    )

    # Create content box with additional text
    content_box = widgets.VBox([widgets.HTML("<center><h1>GreenSat</h1></center>"), instructions])

    # Combine sidebar and content box
    page_content = widgets.HBox([sidebar, content_box])

    return page_content


In [12]:
def page1():
    # Define buttons
    home_button = widgets.Button(description='Home')
    home_button.on_click(home_button_clicked)
    page1_button = widgets.Button(description='Timelapse')
    page1_button.on_click(page1_button_clicked)
    page2_button = widgets.Button(description='Graphs')
    page2_button.on_click(page2_button_clicked)

    # Create button box
    button_box = widgets.VBox([home_button, page1_button, page2_button])

    # Create map
    Map = geemap.Map()
    Map.add_basemap('HYBRID')

    # Define widgets
    style = {'description_width': 'initial'}
    
    speed = widgets.IntSlider(description='  Frames per second:', tooltip='Frames per second:', value=1, min=1, max=5, style=style)
    start_year = widgets.IntSlider(description='Start Year:', value=1995, min=1984, max=2023, style=style)
    end_year = widgets.IntSlider(description='End Year:', value=2023, min=1984, max=2023, style=style)
    start_month = widgets.IntSlider(description='Start Month:', value=1, min=1, max=12, style=style)
    end_month = widgets.IntSlider(description='End Month:', value=12, min=1, max=12, style=style)
    nd_options = [
        'Vegetation Index (NDVI)',
        'Water Index (NDWI)',
        'Modified Water Index (MNDWI)',
        'Snow Index (NDSI)',
        'Soil Index (NDSI)',
        'Burn Ratio (NBR)',
    ]
    nd_indices = widgets.Dropdown(options=nd_options, value=None, description='NDI:', style=style)
    create_gif = widgets.Button(description='Submit', button_style='primary', tooltip='Click to create timelapse', style=style)
    output = widgets.Output()

    # Define widget layouts
    sidebar_box = widgets.VBox([button_box, widgets.Label("Page 1")])
    inputs_box = widgets.VBox([speed, start_year, end_year, start_month, end_month, nd_indices, create_gif, output])
    hbox = widgets.HBox([sidebar_box, inputs_box])
    
    # Button click handlers and map functionality
    first_band = None
    second_band = None

    def nd_index_change(change):
        nonlocal first_band
        nonlocal second_band
        if nd_indices.value == 'Burn Ratio (NBR)':
            first_band = 'NIR'
            second_band = 'SWIR2'
        elif nd_indices.value in [
            'Modified Water Index (MNDWI)',
            'Snow Index (NDSI)',
        ]:
            first_band = 'Green'
            second_band = 'SWIR1'
        elif nd_indices.value == 'Soil Index (NDSI)':
            first_band = 'SWIR1'
            second_band = 'NIR'
        elif nd_indices.value == 'Vegetation Index (NDVI)':
            first_band = 'NIR'
            second_band = 'Red'
        elif nd_indices.value == 'Water Index (NDWI)':
            first_band = 'NIR'
            second_band = 'SWIR1'

    nd_indices.observe(nd_index_change, names='value')

    def submit_clicked(b):
        with output:
            output.clear_output()
            if start_year.value > end_year.value:
                print('The end year must be greater than the start year.')
                return
            if start_month.value > end_month.value:
                print('The end month must be greater than the start month.')
                return
            add_progress_bar = start_year.value != end_year.value
            start_date = f'{str(start_month.value).zfill(2)}-01'
            end_date = f'{str(end_month.value).zfill(2)}-30'

            print('Computing...')

            nd_bands = None
            if (first_band is not None) and (second_band is not None):
                nd_bands = [first_band, second_band]

            Map.add_landsat_ts_gif(
                roi=Map.user_roi,
                label="GreenSat",
                start_year=start_year.value,
                end_year=end_year.value,
                start_date=start_date,
                end_date=end_date,
                bands="SWIR1/NIR/Red".split('/'),
                font_color="white",
                frames_per_second=speed.value,
                font_size=30,
                add_progress_bar=add_progress_bar,
                progress_bar_color="blue",
                download=True,
                apply_fmask=True,
                nd_bands=nd_bands,
                nd_threshold=0,
                nd_palette=["black", "blue"],
            )

    create_gif.on_click(submit_clicked)

    # Create page content
    page1_content = widgets.VBox([hbox, Map])

    return page1_content


In [13]:
import csv
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML

# csv file name
input_filename = r"C:\Users\User\Desktop\owid-co2-data.csv"
output_filename = r"C:\Users\User\Desktop\new.csv"

# Read the CSV file
data = pd.read_csv(input_filename)

# Get the list of unique countries from the first column (index 0)
countries = sorted(set(data.iloc[:, 0]))

# Create a figure
fig = go.FigureWidget()

# Function to generate the graph based on the selected country
def generate_graph(country):
    # Filter data for the specified country
    filtered_data = data[data.iloc[:, 0] == country]

    if filtered_data.empty:
        print("No data found for the specified country.")
    else:
        # Select the last 100 rows
        selected_data = filtered_data.tail(100)

        # Clear previous traces
        fig.data = []

        # Add a scatter trace for the cumulative_luc_co2 column
        fig.add_scatter(x=selected_data["year"], y=selected_data["cumulative_luc_co2"], mode='lines', name='Cumulative LUC CO2')

        # Update layout
        fig.update_layout(title="Changes in Cumulative LUC CO2 {country_dropdown.value}",
                          xaxis_title="Year",
                          yaxis_title="Cumulative LUC CO2")

# Dropdown widget for country selection
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
country_dropdown.observe(lambda change: generate_graph(change.new), names='value')

# Define the page2 function
def page2():
    home_button = widgets.Button(description='Home')
    home_button.on_click(home_button_clicked)
    page1_button = widgets.Button(description='Timelapse')
    page1_button.on_click(page1_button_clicked)
    page2_button = widgets.Button(description='Graphs')
    page2_button.on_click(page2_button_clicked)

    button_box = widgets.VBox([home_button, page1_button, page2_button])
    page2_content = widgets.HBox([button_box, widgets.Label("Carbon Dioxide Levels"), country_dropdown])

    # Generate the initial graph
    generate_graph(country_dropdown.value)

    # Display the page2_content
    #display(HTML("<h2>Page 2</h2>"))
    display(page2_content)
    display(fig)

# Display page 2


In [ ]:
# Display home page initially
display(home_page())


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Cumulative LUC CO2',
              'type': 'scatter',
              'uid': 'd6feae80-1a9b-4056-86eb-e320ed9bc119',
              'x': array([1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933,
                          1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
                          1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,
                          1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
                          1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
                          1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
                          1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
                          2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                          2018, 

None